Importing Necessary Liabraries

In [15]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

Configuration

In [16]:
BASE_DIR = r"image folder"
TRAIN_DIR = os.path.join(BASE_DIR, "train dataset")
VAL_DIR = os.path.join(BASE_DIR, "validation dataset")
TEST_DIR = os.path.join(BASE_DIR, "test dataset")
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
CLASS_NAMES = sorted(os.listdir(TRAIN_DIR))  # Automatically detect classes
NUM_CLASSES = len(CLASS_NAMES)

print(f"Detected {NUM_CLASSES} classes: {CLASS_NAMES}")

Detected 15 classes: ['Bear', 'Bird', 'Cat', 'Cow', 'Deer', 'Dog', 'Dolphin', 'Elephant', 'Giraffe', 'Horse', 'Kangaroo', 'Lion', 'Panda', 'Tiger', 'Zebra']


Data Preparation

In [17]:

def load_dataset(path, subset=None):
    try:
        return tf.keras.utils.image_dataset_from_directory(
            path,
            image_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            label_mode='int',
            validation_split=0.2 if subset else None,
            subset=subset,
            seed=42,
            class_names=CLASS_NAMES
        )
    except Exception as e:
        print(f"Error loading dataset from {path}: {str(e)}")
        return None

# Load datasets with validation
train_ds = load_dataset(TRAIN_DIR)
val_ds = load_dataset(VAL_DIR)
test_ds = load_dataset(TEST_DIR)

if not all([train_ds, val_ds, test_ds]):
    raise SystemExit("Failed to load datasets. Check directory structure.")

Found 1361 files belonging to 15 classes.
Found 195 files belonging to 15 classes.
Found 388 files belonging to 15 classes.


Data Augmentation

In [18]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

# Preprocessing pipeline
def preprocess(image, label):
    image = data_augmentation(image)
    image = tf.keras.applications.xception.preprocess_input(image)
    return image, label

# Optimize dataset pipeline
train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE).cache()

val_ds = val_ds.map(lambda x, y: (tf.keras.applications.xception.preprocess_input(x), y))
val_ds = val_ds.prefetch(tf.data.AUTOTUNE).cache()

Model Construction

In [19]:
base_model = tf.keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)

# Freeze base model initially
base_model.trainable = False

# Build custom head
inputs = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs, outputs)

Training Setup

In [20]:
model.compile(
    optimizer=Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint(
        'best_model.keras',
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1
    )
]

Model Training

In [21]:
print("\n=== Initial Training ===")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)


=== Initial Training ===
Epoch 1/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4102 - loss: 1.9540
Epoch 1: val_accuracy improved from -inf to 0.92821, saving model to best_model.keras
43/43 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.4138 - loss: 1.9416 - val_accuracy: 0.9282 - val_loss: 0.3032
Epoch 2/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8273 - loss: 0.5594
Epoch 2: val_accuracy improved from 0.92821 to 0.95385, saving model to best_model.keras
43/43 ━━━━━━━━━━━━━━━━━━━━ 358s 2s/step - accuracy: 0.8276 - loss: 0.5586 - val_accuracy: 0.9538 - val_loss: 0.1959
Epoch 3/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.9044 - loss: 0.3434 
Epoch 3: val_accuracy did not improve from 0.95385
43/43 ━━━━━━━━━━━━━━━━━━━━ 408s 10s/step - accuracy: 0.9044 - loss: 0.3431 - val_accuracy: 0.9282 - val_loss: 0.1765
Epoch 4/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9443 - loss: 0.2134
Epoch 4: val_accuracy did not improve from 0.95385
43/43 ━━━━━━━━━━

Fine-tuning

In [22]:
print("\n=== Fine-tuning ===")
base_model.trainable = True
for layer in base_model.layers[:40]:
    layer.trainable = False

model.compile(
    optimizer=Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    initial_epoch=history.epoch[-1],
    callbacks=callbacks
)


=== Fine-tuning ===
Epoch 9/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.7993 - loss: 0.7971
Epoch 9: val_accuracy did not improve from 0.95385
43/43 ━━━━━━━━━━━━━━━━━━━━ 441s 10s/step - accuracy: 0.8000 - loss: 0.7950 - val_accuracy: 0.9385 - val_loss: 0.1438
Epoch 10/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.9450 - loss: 0.3481
Epoch 10: val_accuracy did not improve from 0.95385
43/43 ━━━━━━━━━━━━━━━━━━━━ 400s 9s/step - accuracy: 0.9452 - loss: 0.3479 - val_accuracy: 0.9538 - val_loss: 0.1505


Model Saving

In [23]:
model.save('final_model.keras')
print("Model saved successfully")

Model saved successfully


Prediction Utility

In [24]:
class AnimalClassifier:
    def __init__(self, model_path='best_model.keras'):
        self.model = tf.keras.models.load_model(model_path)
        self.class_names = CLASS_NAMES
        
    def predict(self, image_path):
        try:
            img = tf.keras.utils.load_img(image_path, target_size=IMG_SIZE)
            img_array = tf.keras.utils.img_to_array(img)
            img_array = tf.expand_dims(img_array, axis=0)
            img_array = tf.keras.applications.xception.preprocess_input(img_array)
            
            preds = self.model.predict(img_array)
            confidence = np.max(preds)
            class_index = np.argmax(preds)
            
            return {
                'class': self.class_names[class_index],
                'confidence': float(confidence),
                'all_predictions': dict(zip(self.class_names, preds[0].tolist()))
            }
        except Exception as e:
            return {'error': str(e)}

Test Prediction

In [25]:
def test_random_prediction():
    classifier = AnimalClassifier()
    all_images = []
    for root, dirs, files in os.walk(VAL_DIR):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                all_images.append(os.path.join(root, file))
    
    if not all_images:
        return "No images found for testing"
    
    test_image = np.random.choice(all_images)
    actual_class = os.path.basename(os.path.dirname(test_image))
    
    result = classifier.predict(test_image)
    
    print(f"\nTest Image: {os.path.basename(test_image)}")
    print(f"Actual Class: {actual_class}")
    if 'class' in result:
        print(f"Predicted: {result['class']} ({result['confidence']:.2%})")
    else:
        print(f"Error: {result.get('error', 'Unknown error')}")

# Run test prediction
test_random_prediction()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step

Test Image: Giraffe_7_4.jpg
Actual Class: Giraffe
Predicted: Giraffe (71.78%)


Verify model architecture

In [26]:
if model:
    model.summary()

# Test with a known image from training set
test_image = next(iter(test_ds.take(1)))[0][0]

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │         7,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,356,231 (237.87 MB)

 Trainable params: 20,218,983 (77.13 MB)

 Non-trainable params: 1,699,280 (6.48 MB)

 Optimizer params: 40,437,968 (154.26 MB)